<a href="https://colab.research.google.com/github/tndhk/learn-python/blob/master/wiki/wiki_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import requests
import time
import json
from bs4 import BeautifulSoup

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


共通処理群

In [ ]:
api_base_url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/ja.wikipedia.org/all-access/all-agents/'
start_date = '20200101'
end_date = '20210610'

In [ ]:
# scrapingするURLを受け取り、BeautifulSoupオブジェクトを返す
def get_html_bs(url):
  response = requests.get(start_url)
  response.encoding = response.apparent_encoding
  bs = BeautifulSoup(response.text, 'html.parser')
  return bs

In [ ]:
# list(wikiのpage名）を受け取り、ひたすらapiを呼ぶ。マナー上、1秒1回送信にしている。
# headerのMLは利用者のアドレス(api使いすぎた時に連絡が来る模様) 
def call_api(input_list):
  json_list = []
  headers = {
    'Accept': 'application/json; charset=utf-8',
    'User-Agent': 'baggio0707@yahoo.co.jp'
    }

  for row in input_list:
    param = row.rstrip()
    url = api_base_url + param + '/daily/' + start_date + '/' + end_date
    res = requests.get(url, headers=headers)
    res = json.loads(res.text)
    json_list.append(res)
    time.sleep(1)
  return json_list

In [ ]:
# jsonをpandasのdata frameに変換し、必要なカラムをCSVに吐き出す
def write_csv_from_df(json_list, output_file):
  df = pd.DataFrame(columns=['project','article','granularity','timestamp','access','agent','views'])

  for i in json_list:
    try:
      df = pd.concat([df, pd.DataFrame(i['items'])])
    except:
      continue

  df = df[['article','timestamp','views']]

  df.to_csv(output_file)

**日本の監督**

In [ ]:
# 監督取得の処理
start_url = 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E3%81%AE%E6%98%A0%E7%94%BB%E7%9B%A3%E7%9D%A3%E4%B8%80%E8%A6%A7'
output_file = '/content/drive/MyDrive/Colab Notebooks/output/director.csv'


In [ ]:
bs = get_html_bs(start_url)

director_url_list = []

parsed = bs.select('div.mw-parser-output')
parsed = parsed[0].select('li')

#for i in range(0,len(parsed)):
for i in range(1,10):
  try:
    a = parsed[i]
    a = a.find('a')['href']
    a = a.replace('/wiki/', '')
    
    if 'redlink=1' in a:
      continue
  
  except:
    continue
  director_url_list.append(a)

In [ ]:
final_list = call_api(director_url_list)

In [ ]:
write_csv_from_df(final_list, output_file)

**日本のテレビドラマ一覧**

In [ ]:
start_url = 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%83%89%E3%83%A9%E3%83%9E%E4%B8%80%E8%A6%A7'
output_file = '/content/drive/MyDrive/Colab Notebooks/output/JapaneseTVDrama.csv'


In [ ]:
bs = get_html_bs(start_url)

japanese_drama_url_list = []

parsed = bs.select('div.mw-parser-output')
parsed = parsed[0].select('li')

#for i in range(0,len(parsed)):
for i in range(0,10):
  a = parsed[i]
  try:
    a = a.find('a')['href']
    a = a.replace('/wiki/', '')
  except:
    continue
  japanese_drama_url_list.append(a)

In [ ]:
final_list = call_api(japanese_drama_url_list)

In [ ]:
write_csv_from_df(final_list, output_file)

**アメリカのテレビドラマ一覧**

In [ ]:
start_url = 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%A1%E3%83%AA%E3%82%AB%E5%90%88%E8%A1%86%E5%9B%BD%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%83%89%E3%83%A9%E3%83%9E%E4%B8%80%E8%A6%A7'
output_file = '/content/drive/MyDrive/Colab Notebooks/output/US_TVDrama.csv'



In [ ]:
bs = get_html_bs(start_url)

us_drama_url_list = []

parsed = bs.select('div.mw-parser-output')
parsed = parsed[0].select('li')

#for i in range(0,10):
for i in range(0,len(parsed)):
  try:
    a = parsed[i]
    a = a.find('a')['href']
    a = a.replace('/wiki/', '')
    
    if 'redlink=1' in a:
      continue
  except:
    continue
  us_drama_url_list.append(a)

In [ ]:
final_list = call_api(us_drama_url_list)

In [ ]:
#print(final_list)
write_csv_from_df(final_list, output_file)

日本のテレビアニメ


In [ ]:
# 2020年代前半
#start_url = 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%82%A2%E3%83%8B%E3%83%A1%E4%BD%9C%E5%93%81%E4%B8%80%E8%A6%A7_(2020%E5%B9%B4%E4%BB%A3_%E5%89%8D%E5%8D%8A)'
#output_file = '/content/drive/MyDrive/Colab Notebooks/output/Japan_anime_2020_1.csv'

#　2010年代後半
#start_url = 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%82%A2%E3%83%8B%E3%83%A1%E4%BD%9C%E5%93%81%E4%B8%80%E8%A6%A7_(2010%E5%B9%B4%E4%BB%A3_%E5%BE%8C%E5%8D%8A)'
#output_file = '/content/drive/MyDrive/Colab Notebooks/output/Japan_anime_2010_2.csv'

# 2010年代前半
start_url = 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%82%A2%E3%83%8B%E3%83%A1%E4%BD%9C%E5%93%81%E4%B8%80%E8%A6%A7_(2010%E5%B9%B4%E4%BB%A3_%E5%89%8D%E5%8D%8A)'
output_file = '/content/drive/MyDrive/Colab Notebooks/output/Japan_anime_2010_1.csv'


In [ ]:
bs = get_html_bs(start_url)

jp_anime_url_list = []

parsed = bs.findAll('table', {'class': 'wikitable'})

for i in range(1):
#for i in range(0,len(parsed)):
  for tr in parsed[i].findAll('tr'):
    try:
      a = tr.find('a')['href']
      a = a.replace('/wiki/', '')
      if  'redlink = 1' in a :
        continue
    except:
      continue

    jp_anime_url_list.append(a)

In [ ]:
final_list = call_api(jp_anime_url_list)

In [ ]:
write_csv_from_df(final_list, output_file)